I'm a student learning ML, and this is my first trial in the competition on kaglle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/facial-keypoints-detection/training.zip
!unzip /kaggle/input/facial-keypoints-detection/test.zip

In [ ]:
# Import librarys
import pandas as pd
import numpy as np
np.random.seed(42)

import numba as nb

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import tensorflow as tf

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load data
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')

In [ ]:
# Look at datas
train.info()
print()
test.info()

In [ ]:
def img_obj_to_float(dataset):
    image = []
    for i in tqdm(range(dataset.shape[0])):
        # Split images
        img = dataset['Image'][i].split(' ')
        # Fill the nulls in image
        img = ['127' if x == '' else x for x in img]
        image.append(img)
    return np.array(image,dtype = 'float').reshape(-1,96,96,1)

x_train = img_obj_to_float(train)
x_test = img_obj_to_float(test)

Now look at an image

In [ ]:
plt.imshow(x_train[0])

In [ ]:
train.loc[[0]]

In [ ]:
# Now we can drop the original image
train.drop('Image', axis = 1, inplace=True)

> # Deal with the nulls

In [ ]:
# Look at the Nulls
train.info()

There are many nulls in almost all colume and appear in pairs, so lets First deal with nulls about eyes

In [ ]:
# take a look at the heatmap of missing values

eyes_features = ['left_eye_center_x','left_eye_center_y',
                'left_eye_inner_corner_x','left_eye_inner_corner_y',
                'left_eye_outer_corner_x','left_eye_outer_corner_y',
                'left_eyebrow_inner_end_x','left_eyebrow_inner_end_y',
                'left_eyebrow_outer_end_x','left_eyebrow_outer_end_y',
                'right_eye_center_x','right_eye_center_y',
                'right_eye_inner_corner_x','right_eye_inner_corner_y',
                'right_eye_outer_corner_x','right_eye_outer_corner_y',
                'right_eyebrow_inner_end_x','right_eyebrow_inner_end_y',
                'right_eyebrow_outer_end_x','right_eyebrow_outer_end_y']
sns.heatmap(train[eyes_features].isna().T, cmap='summer')
plt.title('Heatmap of missing values')

So my idea is to use the locations of the center of eyes and fill the lost colums assuming the lose eyes have an average size.

In [ ]:
train.loc[[4500]]

In [ ]:
# Add some Features
train['left_eye_length_add'] = train['left_eye_outer_corner_x']-train['left_eye_inner_corner_x']
train['right_eye_length_add'] = train['right_eye_inner_corner_x']-train['right_eye_outer_corner_x']
train['left_eyebrow_length_add'] = train['left_eyebrow_outer_end_x']-train['left_eyebrow_inner_end_x']
train['right_eyebrow_length_add'] = train['right_eyebrow_inner_end_x']-train['right_eyebrow_outer_end_x']

In [ ]:
train.loc[[4500]]

In [ ]:
# Fill some of the colums with there own average values
feature_fill_with_mean = ['left_eye_center_x','left_eye_center_y',
                          'right_eye_center_x','right_eye_center_y',
                          'left_eye_outer_corner_y','left_eye_inner_corner_y',
                          'right_eye_inner_corner_y','right_eye_outer_corner_y',
                          'left_eyebrow_outer_end_y','left_eyebrow_inner_end_y',
                          'right_eyebrow_inner_end_y','right_eyebrow_outer_end_y',
                          'left_eye_length_add','right_eye_length_add',
                          'left_eyebrow_length_add','right_eyebrow_length_add']

for feature in feature_fill_with_mean:
    train[feature].fillna(train[feature].mean(), inplace = True)
    train[feature] = train[feature].astype(float)

In [ ]:
train.isnull().sum()

In [ ]:
# Fill some colums by calculation.
train['left_eye_outer_corner_x'].fillna(
    train['left_eye_center_x']+train['left_eye_length_add']/2, 
    inplace = True)
train['left_eye_inner_corner_x'].fillna(
    train['left_eye_center_x']-train['left_eye_length_add']/2, 
    inplace = True)

train['right_eye_outer_corner_x'].fillna(
    train['right_eye_center_x']-train['right_eye_length_add']/2, 
    inplace = True)
train['right_eye_inner_corner_x'].fillna(
    train['right_eye_center_x']+train['right_eye_length_add']/2, 
    inplace = True)


train['left_eyebrow_outer_end_x'].fillna(
    train['left_eye_center_x']+train['left_eyebrow_length_add']/2, 
    inplace = True)
train['left_eyebrow_inner_end_x'].fillna(
    train['left_eye_center_x']-train['left_eyebrow_length_add']/2, 
    inplace = True)

train['right_eyebrow_outer_end_x'].fillna(
    train['right_eye_center_x']-train['right_eyebrow_length_add']/2, 
    inplace = True)
train['right_eyebrow_inner_end_x'].fillna(
    train['right_eye_center_x']+train['right_eyebrow_length_add']/2, 
    inplace = True)

In [ ]:
train.loc[[4500]]

In [ ]:
train.isnull().sum()

Now we start to deal with mouth. Since we have more mouth_center_bottom_lip, we use this as a reference point and fill up other colums

In [ ]:
# take a look at the heatmap of missing values

mouth_features = ['mouth_left_corner_x','mouth_left_corner_y',
                  'mouth_right_corner_x','mouth_right_corner_y',
                  'mouth_center_top_lip_x','mouth_center_top_lip_y',
                  'mouth_center_bottom_lip_x','mouth_center_bottom_lip_y']
sns.heatmap(train[mouth_features].isna().T, cmap='summer')
plt.title('Heatmap of missing values')

In [ ]:
train['mouth_length_add'] = train['mouth_left_corner_x']-train['mouth_right_corner_x']
train['mouth_hight_add'] = train['mouth_center_bottom_lip_y']-train['mouth_center_top_lip_y']

In [ ]:
feature_fill_with_mean = ['mouth_center_bottom_lip_x','mouth_center_bottom_lip_y','mouth_length_add','mouth_hight_add']
for feature in feature_fill_with_mean:
    train[feature].fillna(train[feature].mean(), inplace = True)
    train[feature] = train[feature].astype(float)

In [ ]:
train['mouth_left_corner_x'].fillna(train['mouth_center_bottom_lip_x']+train['mouth_length_add']/2,inplace = True)
train['mouth_right_corner_x'].fillna(train['mouth_center_bottom_lip_x']-train['mouth_length_add']/2,inplace = True)
train['mouth_left_corner_y'].fillna(train['mouth_center_bottom_lip_y']+train['mouth_hight_add']/2,inplace = True)
train['mouth_right_corner_y'].fillna(train['mouth_center_bottom_lip_y']+train['mouth_hight_add']/2,inplace = True)
train['mouth_center_top_lip_x'].fillna(train['mouth_center_bottom_lip_x'],inplace = True)
train['mouth_center_top_lip_y'].fillna(train['mouth_center_bottom_lip_y']-train['mouth_hight_add'],inplace = True)

In [ ]:
train.isnull().sum()

Now all the nulls are filled

In [ ]:
# Drop all we added
train = train.drop(['left_eye_length_add','right_eye_length_add',
           'left_eyebrow_length_add','right_eyebrow_length_add',
           'mouth_length_add','mouth_hight_add'],axis=1)

In [ ]:
train.info()

In [ ]:
def dataframe_to_array(dataset):
    array = []
    for i in tqdm(range(dataset.shape[0])):
        item = dataset.iloc[i,:]
        array.append(item)
    return np.array(array,dtype = 'float')

In [ ]:
y_train = dataframe_to_array(train)

# Preprocess

In [ ]:
# Normalization
x_train = x_train/255.
x_test = x_test/255.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2, random_state=42)

# Modeling

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image 
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
batch_size = 256
epochs = 50
history = model.fit_generator(datagen.flow(x_train,y_train,batch_size = batch_size)
                    ,epochs = epochs,validation_data=(x_val,y_val),
                    verbose = 1,steps_per_epoch=x_train.shape[0] // batch_size,
                    callbacks=[learning_rate_reduction])

# Predicts

In [ ]:
plt.imshow(x_test[0])

In [ ]:
pred = model.predict(x_test)

# Create submission

In [ ]:
lookid_data = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
rowid = lookid_data['RowId']
rowid=list(rowid)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('face_key_detection_submission.csv',index = False)